# CRITEO SPONSORED SEARCH CONVERSION LOG DATASET

## WHAT IS THIS NOTEBOOK ABOUT

This notebook presents EDA (Exploratory Data Analysis), elements of feature engineering and correlation test of the "CRITEO SPONSORED SEARCH CONVERSION LOG DATASET"

## CONTENTS

1. INTRODUCTION
2. EXPLORATORY DATA ANALYSIS
3. TESTING HYPOTHESES
4. FEATURE ENGINEERING
5. STATISTICAL TESTING OF DESCRIBING FEATURES
6. COMPETITION METRIC
7. SUMMARY
8. LITERATURE


# 1. INTRODUCTION

## DESCRIPTION OF THE DATASET

Criteo Sponsored Search Conversion Log Dataset contains logs obtained from Criteo Predictive Search. \
Each row in the dataset represents an action performed by the user on a product related advertisement. 

### Data description

- Sale : Indicates 1 if conversion occurred and 0 if not.
- SalesAmountInEuro : Indicates the revenue obtained when a conversion took place. This might be different from product-price, due to attribution issues. It is -1, when no conversion took place.
- Time_delay_for_conversion : This indicates the time between click and conversion. It is -1, when no conversion took place.

- click_timestamp: Timestamp of the click. The dataset is sorted according to timestamp.
- nb_clicks_1week: Number of clicks the product related advertisement has received in the last 1 week.
- product_price: Price of the product shown in the advertisement.
- product_age_group: The intended user age group of the user, the product is made for.
- device_type: This indicates whether it is a returning user or a new user on mobile, tablet or desktop. 
- audience_id:  We do not disclose the meaning of this feature.
- product_gender: The intended gender of the user, the product is made for.
- product_brand: Categorical feature about the brand of the product.
- product_category(1-7): Categorical features associated to the product. We do not disclose the meaning of these features.
- product_country: Country in which the product is sold.
- product_id: Unique identifier associated with every product.
- product_title: Hashed title of the product.
- partner_id: Unique identifier associated with the seller of the product.
- user_id: Unique identifier associated with every user.

**All categorical features have been hashed**, **-1 is the missing value indicator**

For more information about the dataset head over to https://ailab.criteo.com/criteo-sponsored-search-conversion-log-dataset/

## IMPORTS FOR THE NOTEBOOK

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

from matplotlib import pyplot as plt
import seaborn as sns


%matplotlib inline

## CONSTANTS
Constants that will be used later in the notebook

In [2]:
PARTNER_ID = 'BD01BAFAE73CF38C403978BBB458300C'

ALL_COLUMN_NAMES = ['Sale', 'SalesAmountInEuro', 'time_delay_for_conversion', 'click_timestamp',
                    'nb_clicks_1week', 'product_price', 'product_age_group', 'device_type',
                    'audience_id', 'product_gender', 'product_brand',
                    'product_category(1)', 'product_category(2)', 'product_category(3)', 'product_category(4)',
                    'product_category(5)', 'product_category(6)', 'product_category(7)',
                    'product_country', 'product_id', 'product_title', 'partner_id', 'user_id']

OBJECT_TYPE_COLUMN_NAMES = ['product_age_group', 'device_type','audience_id', 'product_gender', 'product_brand',
                       'product_category(1)', 'product_category(2)', 'product_category(3)', 'product_category(4)',
                       'product_category(5)', 'product_category(6)', 'product_category(7)',
                       'product_country', 'product_id', 'product_title', 'user_id']

## CREATE CSV - CHOOSE THE PARTNER_ID

The analysis will be performed only for one of many *partner_id* from the dataset. Because the dataset can be considered a big data source, we will create a csv file containing only the information related to chosen ` PARTNER_ID `. Reading the *CriteoSearchData* in chunks we will filter out the rows which interest us.

**WARNING** \
If you already have csv file containing the data, you can skip this step.

In [9]:
df = pd.DataFrame()
for chunk in pd.read_csv("CriteoSearchData", names=ALL_COLUMN_NAMES, sep='\t', low_memory=False, chunksize=100_000):
    filtered_chunk = chunk[chunk['partner_id'] == PARTNER_ID]
    df = pd.concat([df, filtered_chunk])

## READ CSV

In [15]:
filepath = f"CriteoSearchData_{PARTNER_ID}.csv"

if os.path.isfile(filepath):
    df_raw = pd.read_csv(f"CriteoSearchData_{PARTNER_ID}.csv", low_memory=False)
else:
    print('Sorry. You have to create an appropriate csv file first.')

# drop partner_id column -> same for every row
df_raw.drop(labels='partner_id', inplace=True, axis=1)

Before performing EDA, let's preprocess the data - handle NaNs and hashed values.

First of all let's change all the `-1` values in the dataset to `np.NaN` to a indicate missing value.

## PREPROCESSING

In [16]:
df_nans = df_raw.replace('-1', np.NaN)
df_nans.replace(-1, np.NaN, inplace=True)
df_nans.head()

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,product_category(2),product_category(3),product_category(4),product_category(5),product_category(6),product_category(7),product_country,product_id,product_title,user_id
0,0,NaN,NaN,1598925284,0.0,0.0,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,NaN,1B491180398E2F0390E6A588B3BCE291,...,CD599E5A946D00A81A875E085F7EF1E8,NaN,NaN,NaN,NaN,NaN,57A1D462A03BD076E029CF9310C11FC5,4C6C62203B4CE5AA6DFCF17F2604DC37,3CE3D1623D32FB2A901822910AF72A0B C83584C673752...,D0EBCD4402172AE3AA6FD21FB77BDE84
1,0,NaN,NaN,1598918451,0.0,0.0,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,NaN,1B491180398E2F0390E6A588B3BCE291,...,CD599E5A946D00A81A875E085F7EF1E8,NaN,NaN,NaN,NaN,NaN,57A1D462A03BD076E029CF9310C11FC5,C2D84B4DE610350B73A8A3DA6E2D0386,C83584C67375245584CEBF5DCE5BEE2D 355AE5E66602C...,B5EFEEB5D9029A869FCB029016DE19AD
2,0,NaN,NaN,1598929821,8.0,0.0,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,NaN,C45A9AC6D102ACAEEDF0D6F78636D84A,...,CD599E5A946D00A81A875E085F7EF1E8,NaN,NaN,NaN,NaN,NaN,57A1D462A03BD076E029CF9310C11FC5,5F3D4BDE66E181048D0381476A7B84F7,FB6C8ED382D3EF96E794F2D63C251E0B AA907525367F6...,7F2FBFC1CAFAE576A7414C48C2B46B05
3,0,NaN,NaN,1598934884,1.0,0.0,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,NaN,C45A9AC6D102ACAEEDF0D6F78636D84A,...,77FFC91A6157E50A0A114135F18BB1DB,2E4ED2B95CEF7FD11B282CDFDD56CEA0,NaN,NaN,NaN,NaN,57A1D462A03BD076E029CF9310C11FC5,97886AD67171240EED4487ED7C7CEFB1,3B3F1EFC68D245B766D311E45CD6B955 B88B3EC8972EB...,F4726A9A63129483D211711111DABE82
4,0,NaN,NaN,1598898855,0.0,0.0,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,NaN,1B491180398E2F0390E6A588B3BCE291,...,77FFC91A6157E50A0A114135F18BB1DB,C08050B58F54FA81F4580ECE15296698,067525A83E1875CB1828F4F9ADC752F6,NaN,NaN,NaN,57A1D462A03BD076E029CF9310C11FC5,84609A43AB113DFFA77536B4C3DBE376,493B88AC8225128B0F81F0F7199C0441 C83584C673752...,EAD901328DDDBAB9DD3112DC4DC1160C


Now let's handle the hashed values. First of all let's check datatypes of the columns of `df`

In [5]:
print(df.dtypes)
print('-'*30)
print(df.dtypes.value_counts())

Sale                           int64
SalesAmountInEuro            float64
time_delay_for_conversion      int64
click_timestamp                int64
nb_clicks_1week                int64
product_price                float64
product_age_group             object
device_type                   object
audience_id                   object
product_gender                object
product_brand                 object
product_category(1)           object
product_category(2)           object
product_category(3)           object
product_category(4)           object
product_category(5)           object
product_category(6)           object
product_category(7)            int64
product_country               object
product_id                    object
product_title                 object
user_id                       object
dtype: object
------------------------------
object     15
int64       5
float64     2
dtype: int64


Thanks to this summary we know now that 16 features have `object` type which means they have been hashed. We can simplify every column with `object` type. As long as they are hashed, we do not lose any valuable information and what is more - it will simplify the EDA of the dataset later. \
For example we will transform the `product_gender` column by assigning one number for each category. \
To achieve this we will use `LabelEncoder` from `sklearn.preprocessing` package.

In [31]:
df_nans_encoded = pd.DataFrame()
for col in df_nans.columns:
    if col in OBJECT_TYPE_COLUMN_NAMES:
        encoder = LabelEncoder()
        df_nans_encoded[col] = encoder.fit_transform(df_nans[col])
    else:
        df_nans_encoded[col] = df_nans[col]

df_nans_encoded.head()

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,product_category(2),product_category(3),product_category(4),product_category(5),product_category(6),product_category(7),product_country,product_id,product_title,user_id
0,0,NaN,NaN,1598925284,0.0,0.0,0,1,102,0,...,25,75,64,13,1,0,0,5562,2687,24840
1,0,NaN,NaN,1598918451,0.0,0.0,0,1,102,0,...,25,75,64,13,1,0,0,14210,8716,21668
2,0,NaN,NaN,1598929821,8.0,0.0,0,0,102,2,...,25,75,64,13,1,0,0,6894,11803,15187
3,0,NaN,NaN,1598934884,1.0,0.0,0,0,102,2,...,11,14,64,13,1,0,0,11044,2531,29144
4,0,NaN,NaN,1598898855,0.0,0.0,0,0,102,0,...,11,60,2,13,1,0,0,9614,3209,27993


## CREATE FINAL DATASET

In [32]:
df_nans_encoded.click_timestamp = pd.to_datetime(df_nans_encoded.click_timestamp, unit='s', origin='unix')

In [36]:
df_nans_encoded['day'] = df_nans_encoded.click_timestamp.dt.date

In [40]:
df = df_nans_encoded.groupby(['day', 'product_id']).agg({'Sale' : ['sum', 'size'], 'SalesAmountInEuro': 'sum'})

In [41]:
df.head()

Sale      SalesAmountInEuro
                       sum size               sum
day        product_id                            
2020-08-04 1459          0    4               0.0
           1847          0    2               0.0
           2606          0    2               0.0
           2680          0    2               0.0
           3009          0    2               0.0

In [37]:
df_nans_encoded.head(10)

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,product_category(3),product_category(4),product_category(5),product_category(6),product_category(7),product_country,product_id,product_title,user_id,day
0,0,NaN,NaN,2020-09-01 01:54:44,0.0,0.0,0,1,102,0,...,75,64,13,1,0,0,5562,2687,24840,2020-09-01
1,0,NaN,NaN,2020-09-01 00:00:51,0.0,0.0,0,1,102,0,...,75,64,13,1,0,0,14210,8716,21668,2020-09-01
2,0,NaN,NaN,2020-09-01 03:10:21,8.0,0.0,0,0,102,2,...,75,64,13,1,0,0,6894,11803,15187,2020-09-01
3,0,NaN,NaN,2020-09-01 04:34:44,1.0,0.0,0,0,102,2,...,14,64,13,1,0,0,11044,2531,29144,2020-09-01
4,0,NaN,NaN,2020-08-31 18:34:15,0.0,0.0,0,0,102,0,...,60,2,13,1,0,0,9614,3209,27993,2020-08-31
5,0,NaN,NaN,2020-08-31 18:56:47,319.0,0.0,1,1,94,2,...,75,64,13,1,0,0,17000,11189,19565,2020-08-31
6,0,NaN,NaN,2020-08-31 16:46:11,0.0,0.0,0,2,71,0,...,71,64,13,1,0,0,3231,8360,12841,2020-08-31
7,1,119.0,457035.0,2020-08-31 18:30:51,19.0,119.0,0,1,102,0,...,24,64,13,1,0,0,10483,8755,28632,2020-08-31
8,0,NaN,NaN,2020-09-01 01:00:31,0.0,0.0,0,0,102,0,...,75,64,13,1,0,0,3481,8638,26410,2020-09-01
9,0,NaN,NaN,2020-08-31 21:01:51,145.0,0.0,0,0,102,0,...,75,64,13,1,0,0,9085,2127,20136,2020-08-31


In [14]:
df_nans_encoded[['product_id', 'product_title']].sort_values(by='product_id')

,product_id,product_title
31095,0,6521
66004,0,6521
59382,0,6521
59362,0,6521
26045,0,6521
...,...,...
13417,18643,8651
21820,18644,8651
55157,18644,8651
29232,18645,6646


In [8]:
df_nans_encoded.user_id.nlargest(1)

25456    30499
Name: user_id, dtype: int32

In [9]:
df_nans_encoded.product_id.nlargest(1)

29232    18645
Name: product_id, dtype: int32

In [7]:
(~df['product_category(7)'].isna()).sum()

66674

# 2. EXPLORATORY DATA ANALYSIS

In [7]:
df.head()

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,product_category(3),product_category(4),product_category(5),product_category(6),product_category(7),product_country,product_id,product_title,partner_id,user_id
0,0,-1.0,-1,1598925284,0,0.0,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,-1,1B491180398E2F0390E6A588B3BCE291,...,-1,-1,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,4C6C62203B4CE5AA6DFCF17F2604DC37,3CE3D1623D32FB2A901822910AF72A0B C83584C673752...,BD01BAFAE73CF38C403978BBB458300C,D0EBCD4402172AE3AA6FD21FB77BDE84
1,0,-1.0,-1,1598918451,0,0.0,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,-1,1B491180398E2F0390E6A588B3BCE291,...,-1,-1,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,C2D84B4DE610350B73A8A3DA6E2D0386,C83584C67375245584CEBF5DCE5BEE2D 355AE5E66602C...,BD01BAFAE73CF38C403978BBB458300C,B5EFEEB5D9029A869FCB029016DE19AD
2,0,-1.0,-1,1598929821,8,0.0,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,C45A9AC6D102ACAEEDF0D6F78636D84A,...,-1,-1,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,5F3D4BDE66E181048D0381476A7B84F7,FB6C8ED382D3EF96E794F2D63C251E0B AA907525367F6...,BD01BAFAE73CF38C403978BBB458300C,7F2FBFC1CAFAE576A7414C48C2B46B05
3,0,-1.0,-1,1598934884,1,0.0,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,C45A9AC6D102ACAEEDF0D6F78636D84A,...,2E4ED2B95CEF7FD11B282CDFDD56CEA0,-1,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,97886AD67171240EED4487ED7C7CEFB1,3B3F1EFC68D245B766D311E45CD6B955 B88B3EC8972EB...,BD01BAFAE73CF38C403978BBB458300C,F4726A9A63129483D211711111DABE82
4,0,-1.0,-1,1598898855,0,0.0,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,1B491180398E2F0390E6A588B3BCE291,...,C08050B58F54FA81F4580ECE15296698,067525A83E1875CB1828F4F9ADC752F6,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,84609A43AB113DFFA77536B4C3DBE376,493B88AC8225128B0F81F0F7199C0441 C83584C673752...,BD01BAFAE73CF38C403978BBB458300C,EAD901328DDDBAB9DD3112DC4DC1160C


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66674 entries, 0 to 66673
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Sale                       66674 non-null  int64  
 1   SalesAmountInEuro          66674 non-null  float64
 2   time_delay_for_conversion  66674 non-null  int64  
 3   click_timestamp            66674 non-null  int64  
 4   nb_clicks_1week            66674 non-null  int64  
 5   product_price              66674 non-null  float64
 6   product_age_group          66674 non-null  object 
 7   device_type                66674 non-null  object 
 8   audience_id                66674 non-null  object 
 9   product_gender             66674 non-null  object 
 10  product_brand              66674 non-null  object 
 11  product_category(1)        66674 non-null  object 
 12  product_category(2)        66674 non-null  object 
 13  product_category(3)        66674 non-null  obj

As we can see, we do not have any null values in the dataset. However nulls are indicated by `-1`. Let's change them to `np.nan` to find out how much data is really missing.

In [ ]:
df.replace('-1', np.NaN)

In [9]:
df.dtypes.value_counts()

object     16
int64       5
float64     2
dtype: int64

In [8]:
df.describe()

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_category(7)
count,66674.000000,66674.000000,6.667400e+04,6.667400e+04,66674.000000,66674.000000,66674.0
mean,0.079881,13.912320,3.354600e+04,1.599930e+09,21.403606,8.881693,-1.0
std,0.271111,91.153552,2.170364e+05,2.254931e+06,76.542756,40.431734,0.0
min,0.000000,-1.000000,-1.000000e+00,1.596581e+09,-1.000000,0.000000,-1.0
25%,0.000000,-1.000000,-1.000000e+00,1.597817e+09,0.000000,0.000000,-1.0
50%,0.000000,-1.000000,-1.000000e+00,1.599856e+09,0.000000,0.000000,-1.0
75%,0.000000,-1.000000,-1.000000e+00,1.601699e+09,6.000000,0.000000,-1.0
max,1.000000,5062.000000,2.566603e+06,1.604301e+09,821.000000,1040.000000,-1.0


# 3. TESTING HYPOTHESES

# 4. FEATURE ENGINEERING

# 5. STATISTICAL TESTING OF DESCRIBING FEATURES

# 6. COMPETITION METRIC

# 7. SUMMARY